In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [3]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [4]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during training
    """    
    with tf.device(device):

        # Compute the loss like we did in Part II
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0


        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:

                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {:6d}, Epoch {:2d}, Loss: {:.4f}, Accuracy: {:.2%}, Val Loss: {:.4f}, Val Accuracy: {:.2%}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             #train_accuracy.result()*100,
                                             train_accuracy.result(),
                                             val_loss.result(),
                                             #val_accuracy.result()*100))
                                             val_accuracy.result()))
                        #print('Epoch: ', epoch+1)
                        #print('  Iteration: {}'.format(t))
                        #print('  train      loss:      {:.3f}'.format(train_loss.result()))
                        #print('             accuracy:  {:.3f}'.format(train_accuracy.result()*100))
                        #print('  validation loss:      {:.3f}'.format(val_loss.result()))
                        #print('             accuracy:  {:.3f}'.format(val_accuracy.result()*100))

                    t += 1


In [14]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        reg = 0     # 0.0001
        input_shape = (32, 32, 3)
        channel_1, channel_2, channel_3, channel_4, num_classes = 64, 32, 16, 8, 10
        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
    
        self.act = tf.keras.layers.Activation(activation='relu')


        self.conv1 = tf.keras.layers.Conv2D(input_shape = (32, 32, 3),
                                            filters = channel_1, kernel_size = 3, strides = 1, padding = 'same',
                                            use_bias=True,
                                            kernel_initializer = initializer, bias_initializer = tf.zeros_initializer,
                                            kernel_regularizer = tf.keras.regularizers.l2(l=reg))
        
        self.bn = tf.keras.layers.BatchNormalization(axis=-1) # spatial batch norm sur C, tf format NHWC
        self.bn1 = tf.keras.layers.BatchNormalization(axis=-1) # spatial batch norm sur C, tf format NHWC
        
        self.conv1_ = tf.keras.layers.Conv2D(filters = channel_1, kernel_size = 3, strides = 1, padding = 'same',
                                             use_bias=True,
                                             kernel_initializer = initializer, bias_initializer = tf.zeros_initializer,
                                             kernel_regularizer = tf.keras.regularizers.l2(l=reg))
        
        self.bn1_ = tf.keras.layers.BatchNormalization(axis=-1) # spatial batch norm sur C, tf format NHWC
        
        #self.ln = tf.keras.layers.LayerNormalization(axis=3)
        #self.ln = tf.keras.layers.LayerNormalization(axis=3)
        

        self.conv2 = tf.keras.layers.Conv2D(filters = channel_2, kernel_size = 3, strides = 1, padding = 'same',
                                            use_bias=True,
                                            kernel_initializer = initializer, bias_initializer = tf.zeros_initializer,
                                            kernel_regularizer = tf.keras.regularizers.l2(l=reg))
        self.bn2 = tf.keras.layers.BatchNormalization(axis=-1)

        self.conv2_ = tf.keras.layers.Conv2D(filters = channel_2, kernel_size = 3, strides = 1, padding = 'same',
                                            use_bias=True,
                                            kernel_initializer = initializer, bias_initializer = tf.zeros_initializer,
                                            kernel_regularizer = tf.keras.regularizers.l2(l=reg))
        self.bn2_ = tf.keras.layers.BatchNormalization(axis=-1)
        
        self.conv3 = tf.keras.layers.Conv2D(filters = channel_3, kernel_size = 3, strides = 1, padding = 'same',
                                            use_bias=True,
                                            kernel_initializer = initializer, bias_initializer = tf.zeros_initializer,
                                            kernel_regularizer = tf.keras.regularizers.l2(l=reg))
        self.bn3 = tf.keras.layers.BatchNormalization(axis=-1)
        
        self.conv3_ = tf.keras.layers.Conv2D(filters = channel_3, kernel_size = 3, strides = 1, padding = 'same',
                                            use_bias=True,
                                            kernel_initializer = initializer, bias_initializer = tf.zeros_initializer,
                                            kernel_regularizer = tf.keras.regularizers.l2(l=reg))
        self.bn3_ = tf.keras.layers.BatchNormalization(axis=-1)
        
        self.conv4 = tf.keras.layers.Conv2D(filters = channel_4, kernel_size = 3, strides = 1, padding = 'same',
                                            use_bias=True,
                                            kernel_initializer = initializer, bias_initializer = tf.zeros_initializer,
                                            kernel_regularizer = tf.keras.regularizers.l2(l=reg))
        self.bn4 = tf.keras.layers.BatchNormalization(axis=-1)

        self.conv4_ = tf.keras.layers.Conv2D(filters = channel_4, kernel_size = 3, strides = 1, padding = 'same',
                                            use_bias=True,
                                            kernel_initializer = initializer, bias_initializer = tf.zeros_initializer,
                                            kernel_regularizer = tf.keras.regularizers.l2(l=reg))
        self.bn4_ = tf.keras.layers.BatchNormalization(axis=-1)

        #self.conv4 = tf.keras.layers.Conv2D(filters = channel_4, kernel_size = 3, strides = 1, padding = 'same',
        #                                    activation='relu', use_bias=True,
        #                                    kernel_initializer = initializer, bias_initializer = tf.zeros_initializer)
        
        self.pooling = tf.keras.layers.MaxPooling2D()

        self.flatten = tf.keras.layers.Flatten()

        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
    
    
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


        x = self.conv1(input_tensor)
        x = self.bn1(x, training)
        #x = self.ln(x, training)
        x = self.act(x)
        
        x = self.conv1_(x)
        x = self.bn1_(x, training)
        x = self.act(x)        
        
        x = self.pooling(x)
        
        x = self.conv2(x)
        x = self.bn2(x, training)
        x = self.act(x)
        
        x = self.conv2_(x)
        x = self.bn2_(x, training)
        x = self.act(x)
        
        x = self.pooling(x)

        x = self.conv3(x)
        x = self.bn3(x, training)
        x = self.act(x)
        
        #x = self.conv3_(x)
        #x = self.bn3_(x, training)
        #x = self.act(x)
        
        #x = self.pooling(x)
        
        x = self.conv4(x)
        x = self.bn4(x, training)
        x = self.act(x)
        
        #x = self.conv4_(x)
        #x = self.bn4_(x, training)
        #x = self.act(x)
        
        x = self.pooling(x)
        
        x = self.flatten(x)
        x = self.fc(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        
        return x

device = '/device:GPU:0'   # Change this to a CPU/GPU as you wish!
#device = '/cpu:0'        # Change this to a CPU/GPU as you wish!

print_every = 200
num_epochs  = 5

#print_every = 100
#num_epochs  = 1

#print_every = 300
#num_epochs  = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 2e-3
    #learning_rate = 5e-4
    return tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.99,  decay=1e-6) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration      0, Epoch  1, Loss: 3.3116, Accuracy: 15.62%, Val Loss: 3.2062, Val Accuracy: 11.50%
Iteration    200, Epoch  1, Loss: 1.7823, Accuracy: 35.18%, Val Loss: 1.6950, Val Accuracy: 40.90%
Iteration    400, Epoch  1, Loss: 1.6142, Accuracy: 41.28%, Val Loss: 1.3560, Val Accuracy: 52.30%
Iteration    600, Epoch  1, Loss: 1.4961, Accuracy: 45.57%, Val Loss: 1.3468, Val Accuracy: 53.10%
Iteration    800, Epoch  2, Loss: 1.0876, Accuracy: 61.21%, Val Loss: 1.2033, Val Accuracy: 57.70%
Iteration   1000, Epoch  2, Loss: 1.0528, Accuracy: 62.27%, Val Loss: 1.3313, Val Accuracy: 51.20%
Iteration   1200, Epoch  2, Loss: 1.0163, Accuracy: 63.83%, Val Loss: 1.1686, Val Accuracy: 58.60%
Iteration   1400, Epoch  2, Loss: 0.9878, Accuracy: 64.88%, Val Loss: 0.9862, Val Accuracy: 64.60%
Iteration   1600, Epoch  3, Loss: 0.8472, Accuracy: 69.61%, Val Loss: 1.0631, Val Accuracy: 63.80%
Iteration   1800, Epoch  3, Loss: 0.8533, Accuracy: 69.84%, Val Loss: 1.0680, Val Accuracy: 62.50%
Iteration 

    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.99,  decay=1e-6) 
    Iteration    300, Epoch  1, Loss: 2.0678, Accuracy: 23.60%, Val Loss: 1.7464, Val Accuracy: 35.10%
    Iteration   7500, Epoch 10, Loss: 0.8114, Accuracy: 71.25%, Val Loss: 0.9961, Val Accuracy: 66.90%
    pooling on every layer